In [1]:
!pip install -qU boto3 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.33 requires botocore==1.34.33, but you have botocore 1.34.72 which is incompatible.


In [2]:
import json
import os
import sys
import boto3
import base64
import requests
import time
import pprint
from datetime import datetime

In [3]:
#get modelARN
region = 'us-east-1'#'us-west-2' #
boto3_bedrock = boto3.client('bedrock',region)
boto3_bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '77ea9f40-562e-405c-ba8c-796dd2b8590d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 28 Mar 2024 03:17:08 GMT',
   'content-type': 'application/json',
   'content-length': '18333',
   'connection': 'keep-alive',
   'x-amzn-requestid': '77ea9f40-562e-405c-ba8c-796dd2b8590d'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0',
   'modelId': 'amazon.titan-image-generator-v1:0',
   'modelName': 'Titan Image Generator G1',
   'providerName': 'Amazon',

In [4]:
!wget https://cats.com/wp-content/uploads/2020/10/tabby-maine-coon-768x384.jpg

--2024-03-28 03:17:39--  https://cats.com/wp-content/uploads/2020/10/tabby-maine-coon-768x384.jpg
Resolving cats.com (cats.com)... 104.22.46.85, 172.67.39.47, 104.22.47.85, ...
Connecting to cats.com (cats.com)|104.22.46.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45635 (45K) [image/jpeg]
Saving to: ‘tabby-maine-coon-768x384.jpg’

100%[======================================>] 45,635      --.-K/s   in 0s      

2024-03-28 03:17:39 (125 MB/s) - ‘tabby-maine-coon-768x384.jpg’ saved [45635/45635]



In [5]:
stream = False
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [7]:
#"modelId": "anthropic.claude-3-sonnet-20240229-v1:0"
# non streaming mode
def anthropic_claude_3(modelId,image_path,max_tokens):
    bedrock_runtime = boto3.client('bedrock-runtime')
    base64_image = encode_image(image_path)
    payload = {
        "modelId": modelId,
        "contentType": "application/json",
        "accept": "application/json",
        "body": {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": "image/png",
                                "data": base64_image
                            }
                        },
                        {
                            "type": "text",
                            #"text": "Write me a detailed description of these two photos, and then a poem talking about it."
                            "text": f"What’s in this image? output {max_tokens} tokens"
                        }
                    ]
                }
            ]
        }
    }
	
    # Convert the payload to bytes
    body_bytes = json.dumps(payload['body']).encode('utf-8')
	
    # Invoke the model
    response = bedrock_runtime.invoke_model(
        body=body_bytes,
        contentType=payload['contentType'],
        accept=payload['accept'],
        modelId=payload['modelId']
    )
	
    # Process the response
    response_body = json.loads(response['body'].read().decode('utf-8'))
    pprint.pprint(response_body)
    #return round(end-start,2),response_body["usage"]["input_tokens"],response_body["usage"]["output_tokens"]
    return round(float(response['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-invocation-latency'])/1000,2),int(response['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-input-token-count']),int(response['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-output-token-count'])
	

In [8]:
#"modelId": "anthropic.claude-3-sonnet-20240229-v1:0"
# streaming mode
def anthropic_claude_3_stream(modelId,image_path,max_tokens):
    bedrock_runtime = boto3.client('bedrock-runtime')
    base64_image = encode_image(image_path)
    payload = {
        "modelId": modelId,
        "contentType": "application/json",
        "accept": "application/json",
        "body": {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": "image/png",
                                "data": base64_image
                            }
                        },
                        {
                            "type": "text",
                            #"text": "Write me a detailed description of these two photos, and then a poem talking about it."
                            "text": f"What’s in this image? output {max_tokens} tokens"
                        }
                    ]
                }
            ],
            "temperature": 1,
            "top_p": 0.999,
            "top_k": 250,
"stop_sequences": ['\n\nHuman:']
        }
    }
	
    # Convert the payload to bytes
    body_bytes = json.dumps(payload['body']).encode('utf-8')
	
    # Invoke the model
    response = bedrock_runtime.invoke_model_with_response_stream(
        body=body_bytes, modelId=payload['modelId'], accept=payload['accept'], contentType=payload['contentType']
    )
    stream = response.get('body')
    chunk_obj = {}
	
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                pprint.pprint(chunk_obj)
	
    # Process the response
    #response_body = json.loads(response['body'].read().decode('utf-8'))
    #pprint.pprint(response_body)
    {'type': 'message_stop', 'amazon-bedrock-invocationMetrics': {'inputTokenCount': 92, 'outputTokenCount': 277, 'invocationLatency': 3679, 'firstByteLatency': 677}}
	
    return round(float(chunk_obj['amazon-bedrock-invocationMetrics']['firstByteLatency'])/1000,2),round(float(chunk_obj['amazon-bedrock-invocationMetrics']['invocationLatency'])/1000,2),chunk_obj['amazon-bedrock-invocationMetrics']['inputTokenCount'],chunk_obj['amazon-bedrock-invocationMetrics']['outputTokenCount']
	

In [11]:
# 调用结果
image_path = "tabby-maine-coon-768x384.jpg"
max_tokens = 200
#haiku
#print("Haiku:")
#modelId = "anthropic.claude-3-haiku-20240307-v1:0"
#print(anthropic_claude_3(modelId,image_path,max_tokens))
#print(anthropic_claude_3_stream(modelId,image_path,max_tokens))	
#sonnet
print("Sonnet:")
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
print(anthropic_claude_3(modelId,image_path,max_tokens))

#print(anthropic_claude_3_stream(modelId,image_path,max_tokens))		

Sonnet:
{'content': [{'text': 'This image depicts a large domestic cat, likely a Maine '
                      'Coon breed, relaxing on what appears to be a cat tree '
                      'or scratching post. The cat has a thick, long-haired '
                      'coat with distinctive tabby markings, featuring shades '
                      'of brown and black stripes against a lighter tan or '
                      "cream undercoat. The cat's eyes are a striking "
                      'golden-yellow color, and its ears are pointed upwards, '
                      "suggesting an alert and attentive posture. The cat's "
                      'large size and tufted paws are characteristic of the '
                      'Maine Coon breed, known for their impressive size and '
                      'hunting heritage. The background shows a beige or tan '
                      'carpeted surface, suggesting an indoor home setting. '
                      'Overall, the image captures th